In [7]:
import pandas as pd
import numpy as np

# 데이터 로드
df = pd.read_csv('data_to_use.csv')

In [8]:
def merge_nmvl_columns(df):
    """
    각 물질별로 NMVL과 NMVL_2 컬럼을 병합하여 NMVL_merge 컬럼 생성
    
    병합 규칙:
    1. NMVL과 NMVL_2 둘 다 있으면: 평균값
    2. NMVL만 있으면: NMVL 값
    3. NMVL_2만 있으면: NMVL_2 값
    4. 둘 다 없으면: 전체 행의 평균값
    """
    
    # 복사본 생성
    df_merged = df.copy()
    
    # 물질별 컬럼 쌍 찾기
    substance_pairs = []
    
    # NMVL 컬럼들 찾기
    nmvl_columns = [col for col in df.columns if col.endswith('_RSLT_NMVL') and not col.endswith('_RSLT_NMVL_2')]
    
    for nmvl_col in nmvl_columns:
        nmvl_2_col = nmvl_col + '_2'
        if nmvl_2_col in df.columns:
            substance_name = nmvl_col.replace('_RSLT_NMVL', '')
            substance_pairs.append({
                'substance': substance_name,
                'nmvl': nmvl_col,
                'nmvl_2': nmvl_2_col,
                'merged': nmvl_col + '_merge'
            })
    
    print(f"병합할 물질별 컬럼 쌍: {len(substance_pairs)}개")
    
    # 각 물질별로 병합 처리
    merge_statistics = []
    
    for pair in substance_pairs:
        substance = pair['substance']
        nmvl_col = pair['nmvl']
        nmvl_2_col = pair['nmvl_2']
        merged_col = pair['merged']
        
        print(f"\n=== {substance} 컬럼 병합 처리 ===")
        
        # 각 컬럼의 데이터 현황 파악
        nmvl_data = df[nmvl_col].dropna()
        nmvl_2_data = df[nmvl_2_col].dropna()
        
        print(f"{nmvl_col}: {len(nmvl_data)}개 데이터 (결측: {len(df) - len(nmvl_data)}개)")
        print(f"{nmvl_2_col}: {len(nmvl_2_data)}개 데이터 (결측: {len(df) - len(nmvl_2_data)}개)")
        
        # 새로운 병합 컬럼 초기화
        df_merged[merged_col] = np.nan
        
        # 각 경우별 카운터
        both_available = 0
        only_nmvl = 0
        only_nmvl_2 = 0
        both_missing = 0
        
        # 행별로 병합 처리
        for idx in df.index:
            nmvl_val = df.loc[idx, nmvl_col]
            nmvl_2_val = df.loc[idx, nmvl_2_col]
            
            # 1. 둘 다 있는 경우: 평균값
            if pd.notna(nmvl_val) and pd.notna(nmvl_2_val):
                df_merged.loc[idx, merged_col] = (nmvl_val + nmvl_2_val) / 2
                both_available += 1
                
            # 2. NMVL만 있는 경우
            elif pd.notna(nmvl_val) and pd.isna(nmvl_2_val):
                df_merged.loc[idx, merged_col] = nmvl_val
                only_nmvl += 1
                
            # 3. NMVL_2만 있는 경우
            elif pd.isna(nmvl_val) and pd.notna(nmvl_2_val):
                df_merged.loc[idx, merged_col] = nmvl_2_val
                only_nmvl_2 += 1
                
            # 4. 둘 다 없는 경우: 나중에 전체 평균으로 처리
            else:
                both_missing += 1
        
        # 전체 평균 계산 (둘 다 없는 경우를 위해)
        data_merged_temp = df_merged[merged_col].dropna()
        if len(data_merged_temp) > 0:
            overall_mean = data_merged_temp.mean()
            
            # 둘 다 없는 경우 전체 평균으로 채우기
            missing_mask = df_merged[merged_col].isna()
            df_merged.loc[missing_mask, merged_col] = overall_mean
            
            print(f"전체 평균값: {overall_mean:.2f} (결측값 대체용)")
        else:
            print("⚠️ 병합할 데이터가 없어서 전체 평균을 계산할 수 없습니다.")
        
        # 통계 정보 저장
        merge_stat = {
            'substance': substance,
            'both_available': both_available,
            'only_nmvl': only_nmvl,
            'only_nmvl_2': only_nmvl_2,
            'both_missing': both_missing,
            'total': len(df)
        }
        merge_statistics.append(merge_stat)
        
        # 병합 결과 출력
        print(f"병합 결과:")
        print(f"  - 둘 다 있음 (평균): {both_available}건")
        print(f"  - NMVL만 있음: {only_nmvl}건")
        print(f"  - NMVL_2만 있음: {only_nmvl_2}건")
        print(f"  - 둘 다 없음 (전체평균): {both_missing}건")
        print(f"  - 총 {len(df)}건 처리 완료")
    
    return df_merged, merge_statistics

def analyze_merge_results(df_original, df_merged, merge_statistics):
    """병합 결과 분석"""
    
    print("\n" + "="*50)
    print("병합 결과 종합 분석")
    print("="*50)
    
    # 전체 통계 테이블
    stats_df = pd.DataFrame(merge_statistics)
    
    print("\n1. 병합 통계 요약:")
    print(stats_df.to_string(index=False))
    
    # 비율 계산
    print("\n2. 병합 비율 분석:")
    for _, stat in stats_df.iterrows():
        substance = stat['substance']
        total = stat['total']
        
        print(f"\n{substance}:")
        print(f"  - 둘 다 있음: {stat['both_available']}건 ({stat['both_available']/total*100:.1f}%)")
        print(f"  - NMVL만: {stat['only_nmvl']}건 ({stat['only_nmvl']/total*100:.1f}%)")
        print(f"  - NMVL_2만: {stat['only_nmvl_2']}건 ({stat['only_nmvl_2']/total*100:.1f}%)")
        print(f"  - 평균값 대체: {stat['both_missing']}건 ({stat['both_missing']/total*100:.1f}%)")
    
    # 병합된 컬럼들의 기본 통계
    print("\n3. 병합된 컬럼들의 기본 통계:")
    merged_columns = [col for col in df_merged.columns if col.endswith('_merge')]
    
    for col in merged_columns:
        substance = col.replace('_RSLT_NMVL_merge', '')
        data = df_merged[col].dropna()
        
        if len(data) > 0:
            print(f"\n{substance}_merge:")
            print(f"  - 평균: {data.mean():.2f}")
            print(f"  - 표준편차: {data.std():.2f}")
            print(f"  - 최소값: {data.min():.2f}")
            print(f"  - 최대값: {data.max():.2f}")
            print(f"  - 데이터 수: {len(data)}개")

def compare_before_after_merge(df_original, df_merged):
    """병합 전후 비교"""
    
    print("\n" + "="*50)
    print("병합 전후 데이터 비교")
    print("="*50)
    
    # 원본 NMVL 컬럼들
    original_nmvl_cols = [col for col in df_original.columns if col.endswith('_RSLT_NMVL') and not col.endswith('_RSLT_NMVL_2')]
    original_nmvl_2_cols = [col for col in df_original.columns if col.endswith('_RSLT_NMVL_2')]
    merged_cols = [col for col in df_merged.columns if col.endswith('_merge')]
    
    print(f"\n원본 컬럼 수: NMVL {len(original_nmvl_cols)}개, NMVL_2 {len(original_nmvl_2_cols)}개")
    print(f"병합 컬럼 수: {len(merged_cols)}개")
    
    # 각 물질별 데이터 수 비교
    for i, merged_col in enumerate(merged_cols):
        if i < len(original_nmvl_cols):
            nmvl_col = original_nmvl_cols[i]
            nmvl_2_col = nmvl_col + '_2'
            
            substance = merged_col.replace('_RSLT_NMVL_merge', '')
            
            original_nmvl_count = df_original[nmvl_col].dropna().count()
            original_nmvl_2_count = df_original[nmvl_2_col].dropna().count() if nmvl_2_col in df_original.columns else 0
            merged_count = df_merged[merged_col].dropna().count()
            
            print(f"\n{substance}:")
            print(f"  - 원본 NMVL: {original_nmvl_count}개")
            print(f"  - 원본 NMVL_2: {original_nmvl_2_count}개")
            print(f"  - 병합 후: {merged_count}개")
            print(f"  - 데이터 증가: {merged_count - max(original_nmvl_count, original_nmvl_2_count)}개")

def export_data_merged(df_merged, filename='data_merged.csv'):
    """병합된 데이터 내보내기"""
    
    # 병합된 컬럼들만 포함하여 내보내기
    merged_columns = [col for col in df_merged.columns if col.endswith('_merge')]
    basic_columns = ['SCHUL_NM', 'ADRCD_NM', 'SCHUL_KND_SC_CODE', 'SEM_STR']
    
    export_columns = basic_columns + merged_columns
    export_df = df_merged[export_columns].copy()
    
    # 컬럼명 정리
    column_rename = {}
    for col in merged_columns:
        substance = col.replace('_RSLT_NMVL_merge', '')
        column_rename[col] = f"{substance}"
    
    export_df = export_df.rename(columns=column_rename)
    
    # CSV 파일로 저장
    export_df.to_csv(filename, index=False, encoding='utf-8-sig')
    print(f"\n병합된 데이터가 '{filename}' 파일로 저장되었습니다.")
    print(f"저장된 컬럼: {list(export_df.columns)}")
    
    return export_df

# 실행 코드
if __name__ == "__main__":
    print("=== NMVL 컬럼 병합 시작 ===\n")
    
    # 1. 컬럼 병합 수행
    df_merged, merge_statistics = merge_nmvl_columns(df)
    
    # 2. 병합 결과 분석
    analyze_merge_results(df, df_merged, merge_statistics)
    
    # 3. 병합 전후 비교
    compare_before_after_merge(df, df_merged)
    
    # 4. 병합된 데이터 내보내기
    exported_df = export_data_merged(df_merged)
    
    print(f"\n=== 병합 완료 ===")
    print("병합된 컬럼들:")
    merged_cols = [col for col in df_merged.columns if col.endswith('_merge')]
    for col in merged_cols:
        print(f"  - {col}")
    
    print(f"\n병합된 데이터프레임 크기: {df_merged.shape}")
    print("병합 규칙이 모두 적용되었습니다.")

# 개별 물질별 분석 함수
def analyze_substance_individually(df_merged, substance_name):
    """특정 물질의 병합 결과 상세 분석"""
    
    merged_col = f"{substance_name}_RSLT_NMVL_merge"
    
    if merged_col not in df_merged.columns:
        print(f"{substance_name} 물질의 병합 컬럼이 존재하지 않습니다.")
        return
    
    print(f"\n=== {substance_name} 상세 분석 ===")
    
    data = df_merged[merged_col].dropna()
    
    if len(data) > 0:
        print(f"데이터 수: {len(data)}개")
        print(f"평균: {data.mean():.2f}")
        print(f"중앙값: {data.median():.2f}")
        print(f"표준편차: {data.std():.2f}")
        print(f"최소값: {data.min():.2f}")
        print(f"최대값: {data.max():.2f}")
        
        # 분포 정보
        print(f"\n분위수:")
        print(f"  25%: {data.quantile(0.25):.2f}")
        print(f"  50%: {data.quantile(0.50):.2f}")
        print(f"  75%: {data.quantile(0.75):.2f}")
    else:
        print("분석할 데이터가 없습니다.")

=== NMVL 컬럼 병합 시작 ===

병합할 물질별 컬럼 쌍: 6개

=== AIR_BACT 컬럼 병합 처리 ===
AIR_BACT_RSLT_NMVL: 1892개 데이터 (결측: 40개)
AIR_BACT_RSLT_NMVL_2: 421개 데이터 (결측: 1511개)
전체 평균값: 336.70 (결측값 대체용)
병합 결과:
  - 둘 다 있음 (평균): 417건
  - NMVL만 있음: 1475건
  - NMVL_2만 있음: 4건
  - 둘 다 없음 (전체평균): 36건
  - 총 1932건 처리 완료

=== FALL_BACT 컬럼 병합 처리 ===
FALL_BACT_RSLT_NMVL: 1179개 데이터 (결측: 753개)
FALL_BACT_RSLT_NMVL_2: 211개 데이터 (결측: 1721개)
전체 평균값: 1.60 (결측값 대체용)
병합 결과:
  - 둘 다 있음 (평균): 209건
  - NMVL만 있음: 970건
  - NMVL_2만 있음: 2건
  - 둘 다 없음 (전체평균): 751건
  - 총 1932건 처리 완료

=== MITE 컬럼 병합 처리 ===
MITE_RSLT_NMVL: 1837개 데이터 (결측: 95개)
MITE_RSLT_NMVL_2: 127개 데이터 (결측: 1805개)
전체 평균값: 17.98 (결측값 대체용)
병합 결과:
  - 둘 다 있음 (평균): 124건
  - NMVL만 있음: 1713건
  - NMVL_2만 있음: 3건
  - 둘 다 없음 (전체평균): 92건
  - 총 1932건 처리 완료

=== RN 컬럼 병합 처리 ===
RN_RSLT_NMVL: 1767개 데이터 (결측: 165개)
RN_RSLT_NMVL_2: 115개 데이터 (결측: 1817개)
전체 평균값: 45.43 (결측값 대체용)
병합 결과:
  - 둘 다 있음 (평균): 107건
  - NMVL만 있음: 1660건
  - NMVL_2만 있음: 8건
  - 둘 다 없음 (전체평균): 157건
  - 총 1932건 처리 완료

=== O3 컬럼 병